# Classifying taxonomy COL_032024 seq data

## GTDB-Tk
https://github.com/Ecogenomics/GTDBTk?tab=readme-ov-file \
GTDB-Tk identifies a core set of genes in the MAG, and use these to compute a species phylogeny

**The gtdb reference database needs 100GB space so do not create conda env in /home directory. Either create the env in /work or /scratch**

I used this opportunity to create a /scratch directory with this documentation: https://docs.unity.rc.umass.edu/documentation/managing-files/hpc-workspace/

In [ ]:
#create scratch directory
ws_allocate gtdb 30

#FOR LATER:to release scratch space when you don't need it anymore
ws_release gtdb

In [ ]:
#INSTALLATION of gtdbtk in /scratch
cd /scratch3/workspace/nikea_ulrich_uml_edu-gtdb
module load conda/latest
mkdir -p /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs
conda create --prefix /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy python=3.8
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy
conda install -c conda-forge -c bioconda gtdbtk=2.4.0 

In [ ]:
#conda package is bundled with a script that will automatically download, and extract the GTDB-Tk reference data, simply run:
download-db.sh
#This will take awhile -- massive database

https://ecogenomics.github.io/GTDBTk/commands/classify_wf.html

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=150G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 48:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/mcav/slurm-gtdb_taxonomy-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

# Set parameters
SAMPLENAME="mcav"
BINPATH="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/binning/${SAMPLENAME}/${SAMPLENAME}_DASTool_bins"
OUTDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/${SAMPLENAME}"
mkdir -p $OUTDIR

#Run gtdb-tk
gtdbtk classify_wf -x fa --skip_ani_screen \
--genome_dir $BINPATH/ --out_dir $OUTDIR/gtdb_out

#This will process all genomes in the directory <my_genomes> using both bacterial and archaeal marker sets and place the results in <output_dir>.
#Genomes must be in FASTA format (gzip with the extension .gz is acceptable)

# JOB-ID:
# bash script file name: /nikea/COL/bash_scripts/Col_gtdb_taxonomy.sh

dlab job ID: 27123105 \
mcav job ID: 27159595 \
ofav job ID: 27159884 \
pstr job ID: 27159918

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=100G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/mcav/slurm-itol-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

SAMPLENAME="mcav"
WORKDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/${SAMPLENAME}/gtdb_out/classify"
OUTDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/${SAMPLENAME}/gtbd_trees_for_vis"
mkdir -p $OUTDIR

#convert all classify trees to itol readable format
gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.bac120.classify.tree.4.tree --output $OUTDIR/itol.gtdbtk.classify.tree.4.tree

gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.bac120.classify.tree.6.tree --output $OUTDIR/itol.gtdbtk.classify.tree.6.tree

gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.bac120.classify.tree.8.tree --output $OUTDIR/itol.gtdbtk.classify.tree.8.tree

gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.backbone.bac120.classify.tree --output $OUTDIR/itol.gtdbtk.backbone.classify.tree

# JOB-ID:27211859
# bash script file name: /nikea/COL/bash_scripts/Col_gtdb2itol_taxonomy.sh

*This above script could be included in the inital gtdbtk script for next use*

copied all bins to: /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/bins

putting all the bins (from all 4 coral species) together so that they are in the same multiple sequence alignment. Added the species name in front to know what species the bin is from. \
will run gtdb-tk again and then take that msa to iqtree

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=150G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 48:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/slurm-gtdb_taxonomy-all-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

# Set parameters
BINPATH='/scratch3/workspace/nikea_ulrich_uml_edu-gtdb/bins'
OUTDIR='/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/'

#Run gtdb-tk
gtdbtk classify_wf -x fa --skip_ani_screen \
--genome_dir $BINPATH/ --out_dir $OUTDIR/gtdb_allbins_out

#This will process all genomes in the directory <my_genomes> using both bacterial and archaeal marker sets and place the results in <output_dir>.
#Genomes must be in FASTA format (gzip with the extension .gz is acceptable)

# JOB-ID: 27941759
# bash script file name: /nikea/COL/bash_scripts/Col_gtdb_taxonomy_all.sh

## IQ-Tree
http://www.iqtree.org/doc/

In [ ]:
#INSTALLATION
module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy
conda install -c bioconda iqtree

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=100G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/slurm-iqtree-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

#set parameters
MSAPATH='/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/gtdb_allbins_out/align'
MSA="gtdbtk.bac120.user_msa.fasta.gz"
OUTDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/iqtree"
mkdir -p $OUTDIR

cd $OUTDIR

#run iqtree
iqtree -s $MSAPATH/$MSA -T AUTO -m TEST -alrt 1000 -B 1000 --prefix all_bins

# JOB-ID: 27949135
# bash script file name: /nikea/COL/bash_scripts/Col_iqtree.sh

## Kraken2 on samples (all seq data)
specifically those that have been identified by Jordan in Ushijima lab as containing high antimicrobial activity
https://github.com/DerrickWood/kraken2/wiki/Manual

In [ ]:
module load conda/latest
conda create --prefix /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/kraken2 python=3.8
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/kraken2
conda install bioconda::kraken2
conda install bioconda/label/cf201901::kraken2

#install bracken in the same environment
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/kraken2
conda install bracken

Using the database already downloaded and built in /project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/standard 

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 48:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/kraken2/slurm-kraken-pstr-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/kraken2

#db already downloaded and built by Brooke
DBNAME="/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/standard"
READS="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/assembly/pstr/final_reads_filtered"
OUTDIR="/scratch3/workspace/nikea_ulrich_uml_edu-gtdb/kraken2"
LISTPATH="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/"
SAMPLELIST="032024_pstr_sampleids.txt"

THREADS=20
KMER_LEN=35 #(default)
READ_LEN=150

#starting with pstr
# classify each set of paired end reads against refseq taxonomic database
while IFS= read -r SAMPLEID; do
kraken2 --db $DBNAME --threads $THREADS --report $OUTDIR/"${SAMPLEID}".kreport2 --report-zero-counts --paired $READS/"${SAMPLEID}"_host_removed_R1.tagged_filter.fastq.gz $READS/"${SAMPLEID}"_host_removed_R2.tagged_filter.fastq.gz > $OUTDIR/"${SAMPLEID}".kraken2
if [ $? -eq 0 ]; then
        echo "kraken2 completed successfully for sample: $SAMPLEID"
    else
        echo "kraken2 encountered an error for sample: $SAMPLEID"
        exit 1
    fi
done < "$LISTPATH/${SAMPLELIST}"
echo "Kraken2: All samples processed successfully."

conda deactivate

# JOB-ID: 30883862
# bash script file name: /bash_scripts/kraken2-pstr_standard.sh

ran this for the others ofav (30924382), mcav (30992591), and dlab (30993197). I ran these on filtered reads (host, human, and symbionts all removed). I had to repair these reads for megahit assembly but resulted in losing some singleton reads, so opted to just run the filtered reads rather than the repaired reads (keeping as many sequences as possible!)
Also ran on just host-removed pstr (30884134) but becuase kraken2 seemed to identify lots on the human/sym filtered reads.

## Bracken
https://github.com/jenniferlu717/Bracken/blob/master/README.md \
abundance estimation from kraken2 output

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 48:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/bracken/slurm-bracken-pstr-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/kraken2

DBNAME="/project/pi_sarah_gignouxwolfsohn_uml_edu/Reference_genomes/ref_databases/standard"
OUTDIR="/scratch3/workspace/nikea_ulrich_uml_edu-gtdb/kraken2/bracken"
mkdir -p $OUTDIR
LISTPATH="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL"
SAMPLELIST="032024_pstr_sampleids.txt"

THREADS=20
KMER_LEN=35 #(default)
READ_LEN=150

# Generate bracken database - just do once
#bracken-build -d ${DBNAME} -t ${THREADS} -k ${KMER_LEN} -l ${READ_LEN} 

#${KRAKEN_DB}`  = location of the built Kraken 1/Kraken 2/KrakenUniq database
#${THREADS}`    = number of threads to use with Kraken and the Bracken scripts
#${KMER_LEN}`   = length of kmer used to build the Kraken database 
    #Kraken 1/KrakenUniq default kmer length = 31
    #Kraken 2 default kmer length = 35
    #Default set in the script is 35. 
#${READ_LEN}`   = the read length of your data e.g., if you are using 100 bp reads, set it to `100`. 
#need taxo.k2d, hash.k2d, and opts.k2d. in kraken2 database

# Abundance Estimation with Bracken
#trying on just pstr first
KRAKENFILES="/scratch3/workspace/nikea_ulrich_uml_edu-gtdb/kraken2"
LEVEL="S"

while IFS= read -r line; do
    # Execute bracken command for each file
    bracken -d "$DBNAME" -i $KRAKENFILES/"$line".kreport2 -o $OUTDIR/"${line%.kreport2}_$LEVEL.bracken" -r "$READ_LEN" -l "$LEVEL"
if [ $? -eq 0 ]; then
        echo "bracken completed successfully for sample: $SAMPLEID"
    else
        echo "bracken encountered an error for sample: $SAMPLEID"
        exit 1
    fi
    done < "$LISTPATH/${SAMPLELIST}"
    
conda deactivate

#${SAMPLE}.kreport - the kraken report generated for a given dataset
#{BRACKEN_OUTPUT_FILE}.bracken - the desired name of the output file to be generated by the code
#The following optional parameters may be specified:
    #${LEVEL} - Default = 'S'. This specifies that abundance estimation will calculate estimated reads for each species. Other possible options are K (kingdom level), P (phylum), C (class), O (order), F (family), and G (genus).
    #${THRESHOLD} - Default = 10. For species classification, any species with <= 10 (or otherwise specified) reads will not receive any additional reads from higher taxonomy levels when distributing reads for abundance estimation. 
    #If another classification level is specified, thresholding will occur at that level.

# JOB-ID: 31001829
# bash script file name: /bash_scripts/bracken-pstr.sh 

ran this on all others

In [ ]:
#To merge tables, I downloaded combine_bracken_outputs.py and put in the same directory as the bracken output
python combine_bracken_outputs.py --files *.bracken --output 032024_COL_SAN_merged_bracken_species.txt 